# Example of DOPER - EV Fleet

### This examples demonstrates the DOPER framework for a three EV fleet with about 150 kW building base load and 100 kW Photovoltaic (PV) generation.

In [1]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
%matplotlib inline

# from IPython.display import HTML
# HTML('''<script>
# code_show=true; 
# function code_toggle() {
#  if (code_show){
#  $('div.input').hide();
#  } else {
#  $('div.input').show();
#  }
#  code_show = !code_show
# } 
# $( document ).ready(code_toggle);
# </script>
# <form action="javascript:code_toggle()">
# <input type="submit" value="Click here to toggle on/off the raw code."></form>
# ''')

### Import DOPER modules

DOPER consists of several modules which are imported here.

In [2]:
from doper import DOPER, get_solver, get_root, standard_report
from doper.models.basemodel import base_model
from doper.models.battery import add_battery
from doper.example import parameter_add_evfleet, ts_inputs, test_default_parameter, ts_inputs_ev_schedule
from doper.plotting import plot_dynamic

### Setup Optimization Model

DOPER passes the inputs as first arguments and parameter as second argument to the optimization model (control_model). In this example two models are loaded:
* the "base_model" with energy balance of on-site generation, e.g. Photovoltaic, and demand, e.g. building base laod
* the "add_battery" model which adds individual batteries or electric vehicles, as defined by the input parameters

An objective function is defined as the sum of energy and demand cost, eventual revenue from exporting energy, and from frequency regulation. The weighting between the objectives is again defined within the parameters of the model.

In [3]:
from pyomo.environ import Objective, minimize

def control_model(inputs, parameter):
    model = base_model(inputs, parameter)
    model = add_battery(model, inputs, parameter)
    
    def objective_function(model):
        return model.sum_energy_cost * parameter['objective']['weight_energy'] \
               + model.sum_demand_cost * parameter['objective']['weight_demand'] \
               + model.sum_export_revenue * parameter['objective']['weight_export']
              
    model.objective = Objective(rule=objective_function, sense=minimize, doc='objective function')
    return model

### Load Example Parameter

The "parameter" dictinoary object includes all parameters of the optimization model. The high-level categories are:
* Battery: parameter of all batteries in the model. This includes number of batteries, initial and final State of Charge, power, capacity, etc.
* Controller: selection of controller options such as model timestep, optimization horizon, etc.
* Objective: weighting of the multiple objectives
* Site: specifications of the site such as timezone, import and export power limits, customer type, etc.
* Tariff: electricity tariff specified as time-of-use with any number of periods

In [4]:
parameter = test_default_parameter()
parameter = parameter_add_evfleet(parameter)

print("parameter 'system' object:")
pprint(parameter['system'])
print('')

print("parameter 'batteries' object:")
pprint(parameter['batteries'])
print('')

parameter 'system' object:
{'battery': True,
 'external_gen': False,
 'genset': False,
 'hvac_control': False,
 'load_control': False,
 'pv': True,
 'reg_bidding': False,
 'reg_response': False}

parameter 'batteries' object:
[{'capacity': 24,
  'efficiency_charging': 0.96,
  'efficiency_discharging': 0.96,
  'power_charge': 15,
  'power_discharge': 15,
  'self_discharging': 0.003,
  'soc_initial': 0.5,
  'soc_max': 1,
  'soc_min': 0},
 {'capacity': 24,
  'efficiency_charging': 0.96,
  'efficiency_discharging': 0.96,
  'power_charge': 15,
  'power_discharge': 15,
  'self_discharging': 0.003,
  'soc_initial': 0.25,
  'soc_max': 1,
  'soc_min': 0},
 {'capacity': 54,
  'efficiency_charging': 0.96,
  'efficiency_discharging': 0.96,
  'power_charge': 30,
  'power_discharge': 30,
  'self_discharging': 0.003,
  'soc_initial': 0.5,
  'soc_max': 1,
  'soc_min': 0}]



### Load Example Data

The input data in this example is chosen to represent a medium-sized office building with rooftop photovoltaic system and three electric vehicles.

In [5]:
data = ts_inputs(parameter, load='B90', scale_load=150, scale_pv=100)
data = ts_inputs_ev_schedule(parameter, data)

### Conduct Optimization

In order to conduct the optimization, the DOPER object is initialized with the model, parameter, and the pyomo_to_pandas converter function. In the next step, the model is evaluated with the "do_optimization" class function. It takes the current inputs as input and returns a list of results.

Note that DOPER is built for application as Model Predictive Control (MPC) where the optimization model is periodically, e.g. every five minutes, updated with the most recent inputs, and reevaluated. However, in this example, do_optimization is only executed once.

In [6]:
# Define the path to the solver executable
solver_path = get_solver('cbc', solver_dir=os.path.join(get_root(), 'solvers'))
print(solver_path)
# Initialize DOPER
smartDER = DOPER(model=control_model,
                 parameter=parameter,
                 solver_path=solver_path)

# Conduct optimization
res = smartDER.do_optimization(data)

# Get results
duration, objective, df, model, result, termination, parameter = res
print(standard_report(res))

/usr/local/lib/python3.8/dist-packages/doper/solvers/Linux64/cbc


KeyError: 'name'

### Result for Site

In [ ]:
plotData = plot_dynamic(df, parameter, plotFile = None, plot_reg=False)